In [1]:
import numpy as np
from scipy.integrate import solve_ivp, odeint

In [2]:
def dUdt(U, t, a, b):
        x = U[:len(a)]
        y = U[len(a):]

        dxdt = a*x - x*y
        dydt = b*x*y - y

        return np.concatenate([dxdt, dydt])

In [3]:
a = np.random.uniform(-10, 10, 1000)
b = np.random.uniform(-10, 10, 1000)
x0, y0 = np.array([0.5] * 1000), np.array([0.5] * 1000)
S0 = np.concatenate([x0, y0])
tspan = np.linspace(0, 10, 100)

In [4]:
sol = odeint(dUdt, S0, tspan, args=(a, b))

In [125]:
import numpy as np 
from sklearn import metrics
from scipy.stats import energy_distance, cramervonmises_2samp
from scipy.special import kl_div
from scipy.spatial.distance import pdist, squareform

def wasserstein_distance(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    # Mean Difference between simulated and observed
    distance = np.mean(np.abs(simulated_sample - observed_sample))
    return distance

def maximum_mean_discrepancy(simulated_sample: np.ndarray, observed_sample: np.ndarray, gamma = 1.0) -> float:
    XX = metrics.pairwise.rbf_kernel(simulated_sample, simulated_sample, gamma)
    YY = metrics.pairwise.rbf_kernel(observed_sample, observed_sample, gamma)
    XY = metrics.pairwise.rbf_kernel(simulated_sample, observed_sample, gamma)

    return XX.mean() + YY.mean() - 2 * XY.mean()

def cramer_von_mises(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    return cramervonmises_2samp(simulated_sample, observed_sample).statistic

def energy_dist(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    n = simulated_sample.shape[1]
    distances_XX = np.zeros((n, 100, 100))  # Distances within array1
    distances_YY = np.zeros((n, 100, 100))  # Distances within array2
    distances_XY = np.zeros((n, 100, 100))  # Distances between array1 and array2

    for i in range(n):
        distances_XX[i] = squareform(pdist(simulated_sample[:, i][:, np.newaxis], metric='euclidean'))
        distances_YY[i] = squareform(pdist(observed_sample[:, i][:, np.newaxis], metric='euclidean'))
        distances_XY[i] = np.linalg.norm(simulated_sample[:, i][:, np.newaxis] - observed_sample[:, i][:, np.newaxis], axis=0)

    mean_dist_XY = np.mean(distances_XY, axis=(1, 2))  # Mean distance between columns
    mean_dist_XX = np.mean(distances_XX, axis=(1, 2))  # Mean distance within array1
    mean_dist_YY = np.mean(distances_YY, axis=(1, 2))  # Mean distance within array2

    # Calculate the energy distances for each column in a vectorized way
    energy_distances = 2 * mean_dist_XY - mean_dist_XX - mean_dist_YY

    # Output the energy distances for each column
    return energy_distances

def kullback_leibler_divergence(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    # Sum of the divergence of each point
    return sum(kl_div(simulated_sample, observed_sample))

In [10]:
sol[:, :len(a)].shape

(100, 1000)

In [122]:
temp

array([2.48952359e+01, 3.38116693e+06, 6.95855273e+09, 1.81559378e+28,
       1.70231214e+11, 2.48369306e+01, 2.48627121e+01, 2.34029274e+01,
       2.47714575e+01, 1.77983911e+27, 2.49349889e+01, 2.48527413e+01,
       2.49201973e+01, 2.37463034e+01, 2.47124058e+01, 4.43436354e+02,
       2.31007204e+01, 3.35059066e+01, 2.46662908e+01, 3.97742064e+02,
       2.49058538e+01, 1.15244706e+42, 2.49204918e+01, 3.40952292e+26,
       1.61523601e+01, 1.57526419e+01, 2.39322137e+09, 2.48597630e+01,
       2.48763666e+01, 2.40483473e+01, 2.49371329e+01, 2.45300565e+01,
       3.14076838e+33, 2.49311550e+01, 2.54616988e+01, 3.23806377e+23,
       2.49201625e+01, 2.49226971e+01, 2.54487147e+01, 7.78003234e+03,
       2.49113973e+01, 1.14155864e+36, 2.49338163e+01, 2.48506523e+01,
       2.47560109e+01, 2.48983209e+01, 5.35251793e+01, 2.49297815e+01,
       2.49358892e+01, 5.30579947e+01, 2.41703506e+01, 3.35878060e+01,
       2.48545599e+01, 2.49281647e+01, 2.87440877e+01, 2.48917139e+01,
      

In [6]:
x_sol = sol[:, :len(a)] # Prey
y_sol = sol[:, len(a):] # Predator

In [13]:
observed = odeint(dUdt, (0.5, 0.5), tspan, args=([1], [1]))

In [33]:
x_observed = observed[:, 0]
x_obs_mod = np.tile(x_observed, (1000, 1)).T
y_observed = observed[:, 1]
y_obs_mod = np.tile(y_observed, (1000, 1)).T

In [126]:
temp = energy_dist(x_sol, x_obs_mod)

In [66]:
x_sol.shape

(100, 1000)